In [1]:
import os, glob, pathlib, shutil, random
from config import Config
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
import _pickle as pk
import warnings
warnings.simplefilter("ignore")

In [2]:
mRS = 42
clfs = {
    "LR": LogisticRegression(random_state=mRS), # Logistic Regression
    "SVC": SVC(random_state=mRS), # Support Vector Classification
    "SGD": SGDClassifier(random_state=mRS), # Stochastic Gradient Descent
    "Perceptron": Perceptron(random_state=mRS), # Perceptron
    "MLP": MLPClassifier(random_state=mRS), #Multi-layer Perceptron
}
cfg  = Config()

In [3]:
def save_raw_clf(kind, clf, cfg):
    print(clf)
    rawClfPath = os.path.join(cfg.Checkpoint, kind + '.pk')
    pathlib.Path(os.path.dirname(rawClfPath)).mkdir(parents=True, exist_ok=True)
    with open(rawClfPath, 'wb') as fclf:
        pk.dump(clf, fclf)

def load_raw_clf(kind, cfg):
    clf = None
    rawClfPath = os.path.join(cfg.Checkpoint, kind + '.pk')
    with open(rawClfPath, 'rb') as fclf:
        clf = pk.load(fclf)
    return clf

In [4]:
print("\nSaving checkpoints for following algorithms...\n")
for kind in clfs.keys():
    save_raw_clf(kind, clfs[kind], cfg)


Saving checkpoints for following algorithms...

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=42,
    shrinking=True, tol=0.001, verbose=False)
SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=